In [1]:
# http://docs.sqlalchemy.org/en/latest/orm/tutorial.html
# https://stackoverflow.com/questions/37806625/sqlalchemy-create-relations-but-without-foreign-key-constraint-in-db
import sqlalchemy
sqlalchemy.__version__ 

'1.2.4'

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [3]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    role_id = Column(Integer)
    
    role = relationship(
        'Role', 
        foreign_keys=[role_id],
        primaryjoin='User.role_id == Role.id'
    )
    
    def __repr__(self):
        return f"<User id: {self.id}; name: {self.name}; role_id: {self.role_id}>"
    
class Role(Base):
    __tablename__ = 'roles'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    

    def __repr__(self):
        return f"<Role id: {self.id}; name: {self.name}>"
    
Base.metadata.create_all(engine)

2018-03-07 19:46:46,433 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-07 19:46:46,433 INFO sqlalchemy.engine.base.Engine ()
2018-03-07 19:46:46,435 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-07 19:46:46,435 INFO sqlalchemy.engine.base.Engine ()
2018-03-07 19:46:46,436 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-03-07 19:46:46,437 INFO sqlalchemy.engine.base.Engine ()
2018-03-07 19:46:46,438 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("roles")
2018-03-07 19:46:46,439 INFO sqlalchemy.engine.base.Engine ()
2018-03-07 19:46:46,440 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	role_id INTEGER, 
	PRIMARY KEY (id)
)


2018-03-07 19:46:46,441 INFO sqlalchemy.engine.base.Engine ()
2018-03-07 19:46:46,442 INFO sqlalchemy.engine.base.Engine COMMIT
2018-03-07 19:46:46,442 INFO sqlalchemy.engine.base.E

In [4]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [5]:
for i in range(10):
    user = User(name=f'u{i}')
    role = Role(name=f'r{i}')
    
    s = Session()

    s.add(role)
    s.flush() 

    user.role_id = role.id

    s.add(user)
    s.commit()
    print('====')

2018-03-07 19:46:46,473 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-07 19:46:46,475 INFO sqlalchemy.engine.base.Engine INSERT INTO roles (name) VALUES (?)
2018-03-07 19:46:46,475 INFO sqlalchemy.engine.base.Engine ('r0',)
2018-03-07 19:46:46,478 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, role_id) VALUES (?, ?)
2018-03-07 19:46:46,479 INFO sqlalchemy.engine.base.Engine ('u0', 1)
2018-03-07 19:46:46,480 INFO sqlalchemy.engine.base.Engine COMMIT
====
2018-03-07 19:46:46,482 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-07 19:46:46,482 INFO sqlalchemy.engine.base.Engine INSERT INTO roles (name) VALUES (?)
2018-03-07 19:46:46,483 INFO sqlalchemy.engine.base.Engine ('r1',)
2018-03-07 19:46:46,484 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, role_id) VALUES (?, ?)
2018-03-07 19:46:46,485 INFO sqlalchemy.engine.base.Engine ('u1', 2)
2018-03-07 19:46:46,487 INFO sqlalchemy.engine.base.Engine COMMIT
====
2018-03-07 19:46:46,489 INFO s

In [6]:
# BAD WAY: 11 SQL QUERYS IN ALL
s = Session()
users = s.query(User).all()
print(users)
for user in users:
    print('====')
    print(user.role)
    print('====')
s.commit()

2018-03-07 19:46:46,561 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-07 19:46:46,563 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.role_id AS users_role_id 
FROM users
2018-03-07 19:46:46,564 INFO sqlalchemy.engine.base.Engine ()
[<User id: 1; name: u0; role_id: 1>, <User id: 2; name: u1; role_id: 2>, <User id: 3; name: u2; role_id: 3>, <User id: 4; name: u3; role_id: 4>, <User id: 5; name: u4; role_id: 5>, <User id: 6; name: u5; role_id: 6>, <User id: 7; name: u6; role_id: 7>, <User id: 8; name: u7; role_id: 8>, <User id: 9; name: u8; role_id: 9>, <User id: 10; name: u9; role_id: 10>]
====
2018-03-07 19:46:46,566 INFO sqlalchemy.engine.base.Engine SELECT roles.id AS roles_id, roles.name AS roles_name 
FROM roles 
WHERE roles.id = ?
2018-03-07 19:46:46,567 INFO sqlalchemy.engine.base.Engine (1,)
<Role id: 1; name: r0>
====
====
2018-03-07 19:46:46,568 INFO sqlalchemy.engine.base.Engine SELECT roles.id AS roles_id, roles.n

In [7]:
# GOOD WAY: ONLY 2 SQL QUERYS
from sqlalchemy.orm import subqueryload
s = Session()

users = s.query(User).options(subqueryload(User.role)).all()
print('====')
print(users)
for user in users:
    print(user.role)
s.commit()

2018-03-07 19:46:46,601 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-07 19:46:46,602 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.role_id AS users_role_id 
FROM users
2018-03-07 19:46:46,603 INFO sqlalchemy.engine.base.Engine ()
2018-03-07 19:46:46,605 INFO sqlalchemy.engine.base.Engine SELECT roles.id AS roles_id, roles.name AS roles_name, anon_1.users_role_id AS anon_1_users_role_id 
FROM (SELECT DISTINCT users.role_id AS users_role_id 
FROM users) AS anon_1 JOIN roles ON anon_1.users_role_id = roles.id ORDER BY anon_1.users_role_id
2018-03-07 19:46:46,606 INFO sqlalchemy.engine.base.Engine ()
====
[<User id: 1; name: u0; role_id: 1>, <User id: 2; name: u1; role_id: 2>, <User id: 3; name: u2; role_id: 3>, <User id: 4; name: u3; role_id: 4>, <User id: 5; name: u4; role_id: 5>, <User id: 6; name: u5; role_id: 6>, <User id: 7; name: u6; role_id: 7>, <User id: 8; name: u7; role_id: 8>, <User id: 9; name: u8; role_id: 9>, <